In [ ]:
import pandas as pd
import numpy as np
import random

from sklearn import model_selection

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.autograd import Variable

# cross validation and metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer


In [ ]:
!pip install -U torchtext==0.8.0

In [ ]:
!pip install transformers

In [ ]:
np.random.seed(500)

In [ ]:
df1 = pd.read_csv("/content/drive/MyDrive/reddit datasets/both_test.csv")
df2 = pd.read_csv("/content/drive/MyDrive/reddit datasets/both_test.csv")
df3 = pd.read_csv("/content/drive/MyDrive/reddit datasets/both_val.csv")
df4 = pd.read_csv("/content/drive/MyDrive/reddit datasets/schizoaffective.csv")
df5 = pd.read_csv("/content/drive/MyDrive/reddit datasets/AlAnon.csv")
df6 = pd.read_csv("/content/drive/MyDrive/reddit datasets/BorderlinePDisorder_10 (1).csv")
df7 = pd.read_csv("/content/drive/MyDrive/reddit datasets/alcoholicsanonymous.csv")
df8 = pd.read_csv("/content/drive/MyDrive/reddit datasets/OCD.csv")

In [ ]:
df4.drop(columns=["score","url","Unnamed: 0","comms_num","created","ups","downs"], inplace=True)
df5.drop(columns=["score","url","Unnamed: 0","comms_num","created","ups","downs"], inplace=True)
df6.drop(columns=["score","url","Unnamed: 0","comms_num","created","ups","downs"], inplace=True)
df7.drop(columns=["score","url","Unnamed: 0","comms_num","created","ups","downs"], inplace=True)
df8.drop(columns=["score","url","Unnamed: 0","comms_num","created","ups","downs"], inplace=True)

In [ ]:
df4 = df4[df4["body"] != "[deleted]"]
df5 = df5[df5["body"] != "[deleted]"]
df6 = df6[df6["body"] != "[deleted]"]
df7 = df7[df7["body"] != "[deleted]"]
df8 = df8[df8["body"] != "[deleted]"]

In [ ]:
df4["class_id"] = [6]*len(df4)
df5["class_id"] = [7]*len(df5)
df6["class_id"] = [8]*len(df6)
df7["class_id"] = [7]*len(df7)
df8["class_id"] = [9]*len(df8)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [ ]:
df4.dropna(inplace=True)
df5.dropna(inplace=True)
df6.dropna(inplace=True)
df7.dropna(inplace=True)
df8.dropna(inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid 

In [ ]:
df4.rename(columns={"body":"post","id":"ID"}, inplace=True)
df5.rename(columns={"body":"post","id":"ID"}, inplace=True)
df6.rename(columns={"body":"post","id":"ID"}, inplace=True)
df7.rename(columns={"body":"post","id":"ID"}, inplace=True)
df8.rename(columns={"body":"post","id":"ID"}, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
frames = [df1, df2, df3, df4, df5, df6, df7, df8]
corpus = pd.concat(frames, ignore_index=True)

In [ ]:
corpus

,ID,title,post,class_name,class_id
0,b07b978c-7760-4932-85cb-5797b5e74168,"basic needs for neurotypicals : food, clothing...",most people able to be productive and function...,adhd,0
1,b2868fd2-e885-400e-b780-446c9581b1f1,"""you're just an iphone in an android world.: d...",i'm 23f and recently quit my job as a features...,adhd,0
2,3191b3b1-d355-41b6-9588-caeb3e169c9b,"""what kind of music do you listen to?"" no answer",i just got finished with an interview and as t...,adhd,0
3,a3d60895-ef7c-4b57-b5be-632c9fd62150,couldn’t help crying while reading a post that...,off course i’ve forgotten what i exactly read....,adhd,0
4,66fae18e-e744-495f-b7a0-b985543f5052,shout-out to all the parents with adhd childre...,"i have six siblings, and at least 4 of us have...",adhd,0
...,...,...,...,...,...
12081,qoprjq,Can you have ocd without having depression???,I mean I do feel very sad and worthless becaus...,NaN,9
12082,qor9f9,"Hi! I'm an OCD therapist, and I'm new to reddit.",Hi OCD Community! I am a clinical psychologist...,NaN,9
12083,qoslls,Vent: my mom who I still live with tested posi...,My mom tested positive for covid. Despite the ...,NaN,9
12084,qotd5d,Do “intrusive memories” count as intrusive tho...,Random anxiety provoking or triggering memorie...,NaN,9


In [ ]:
corpus["text"] = corpus.title + " " + corpus.post

In [ ]:
corpus

,ID,title,post,class_name,class_id,text
0,b07b978c-7760-4932-85cb-5797b5e74168,"basic needs for neurotypicals : food, clothing...",most people able to be productive and function...,adhd,0,"basic needs for neurotypicals : food, clothing..."
1,b2868fd2-e885-400e-b780-446c9581b1f1,"""you're just an iphone in an android world.: d...",i'm 23f and recently quit my job as a features...,adhd,0,"""you're just an iphone in an android world.: d..."
2,3191b3b1-d355-41b6-9588-caeb3e169c9b,"""what kind of music do you listen to?"" no answer",i just got finished with an interview and as t...,adhd,0,"""what kind of music do you listen to?"" no answ..."
3,a3d60895-ef7c-4b57-b5be-632c9fd62150,couldn’t help crying while reading a post that...,off course i’ve forgotten what i exactly read....,adhd,0,couldn’t help crying while reading a post that...
4,66fae18e-e744-495f-b7a0-b985543f5052,shout-out to all the parents with adhd childre...,"i have six siblings, and at least 4 of us have...",adhd,0,shout-out to all the parents with adhd childre...
...,...,...,...,...,...,...
12081,qoprjq,Can you have ocd without having depression???,I mean I do feel very sad and worthless becaus...,NaN,9,Can you have ocd without having depression??? ...
12082,qor9f9,"Hi! I'm an OCD therapist, and I'm new to reddit.",Hi OCD Community! I am a clinical psychologist...,NaN,9,"Hi! I'm an OCD therapist, and I'm new to reddi..."
12083,qoslls,Vent: my mom who I still live with tested posi...,My mom tested positive for covid. Despite the ...,NaN,9,Vent: my mom who I still live with tested posi...
12084,qotd5d,Do “intrusive memories” count as intrusive tho...,Random anxiety provoking or triggering memorie...,NaN,9,Do “intrusive memories” count as intrusive tho...


In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [ ]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(corpus['text'], corpus['class_id'], test_size=0.3, shuffle=True)

In [ ]:
embed_size = 300 # how big is each word vector
max_features = 120000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 750 # max number of words in a question to use
batch_size = 512 # how many samples to process at once
n_splits = 6 # Number of K-fold Splits

In [ ]:
#Tokenize the sentences
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(x_train))
x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)

# Pad the sentences 
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test= pad_sequences(x_test, maxlen=maxlen)

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip
!ls -lat

--2021-12-09 16:39:04--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-12-09 16:39:05--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-12-09 16:39:05--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.1’

glove

In [ ]:
def load_glove(word_index,embed_size):
    EMBEDDING_FILE = '../content/glove.6B.'+str(embed_size)+'d.txt'
    def get_coefs(word,*arr): 
      return word, np.asarray(arr, dtype='float32')[:embed_size]

    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE))
    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = -0.005838499,0.48782197
    embed_size_2 = all_embs.shape[1]
    nb_words = min(max_features, len(word_index)+1)
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size_2))
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
        else:
            embedding_vector = embeddings_index.get(word.capitalize())
            if embedding_vector is not None:
                embedding_matrix[i] = embedding_vector
    return embedding_matrix

embedding_matrix = load_glove(tokenizer.word_index,embed_size)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
np.shape(embedding_matrix)

(30520, 300)

In [ ]:
class TextCNN_Classifier(nn.Module):
    
    def __init__(self):
        super(TextCNN_Classifier, self).__init__()
        #kernel size will be filter _size * embedding size
        # we will have 5 filter covering these many words at a time
        self.filter_1 = 1 
        self.filter_2 = 2
        self.filter_3 = 3
        self.filter_4 = 4
        self.filter_5 = 5
        num_filters = 15 # no of output channels
        number_classes = 10
        self.embedding = nn.Embedding(max_features, embed_size)
        self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = False
        self.conv_1 = nn.Conv2d(1,num_filters,(self.filter_1, embed_size))
        self.conv_2 = nn.Conv2d(1,num_filters,(self.filter_2, embed_size))
        self.conv_3 = nn.Conv2d(1,num_filters,(self.filter_3, embed_size))
        self.conv_4 = nn.Conv2d(1,num_filters,(self.filter_4, embed_size))
        self.conv_5 = nn.Conv2d(1,num_filters,(self.filter_5, embed_size))
        self.dropout = nn.Dropout(0.1)
        self.fc1 = nn.Linear(5*num_filters, number_classes) # number of layers is 5

    def forward(self, x):
        x = self.embedding(x)  
        x =  x.unsqueeze(1)
        x1 =  F.max_pool1d( F.relu(self.conv_1(x)).squeeze(3) , F.relu(self.conv_1(x)).squeeze(3).size(2)).squeeze(2)
        x2 =  F.max_pool1d( F.relu(self.conv_2(x)).squeeze(3) , F.relu(self.conv_2(x)).squeeze(3).size(2)).squeeze(2)
        x3 =  F.max_pool1d( F.relu(self.conv_3(x)).squeeze(3) , F.relu(self.conv_3(x)).squeeze(3).size(2)).squeeze(2)
        x4 =  F.max_pool1d( F.relu(self.conv_4(x)).squeeze(3) , F.relu(self.conv_4(x)).squeeze(3).size(2)).squeeze(2)
        x5 =  F.max_pool1d( F.relu(self.conv_5(x)).squeeze(3) , F.relu(self.conv_5(x)).squeeze(3).size(2)).squeeze(2)
        x = torch.cat((x1,x2,x3,x4,x5),1)
        x = self.dropout(x)
        x = self.fc1(x) 
        return x

In [ ]:
n_epochs = 8
model = TextCNN_Classifier()
loss_fn = nn.CrossEntropyLoss(reduction='sum')
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)
device = torch.device("cpu")
model.to(device)

# Load train and test in CUDA Memory
x_train_2 = torch.tensor((x_train), dtype=torch.long)
y_train_2 = torch.tensor(y_train.values, dtype=torch.long)
x_cv = torch.tensor(x_test, dtype=torch.long)
y_cv = torch.tensor(y_test, dtype=torch.long)

# Create Torch datasets
train = torch.utils.data.TensorDataset(x_train_2, y_train_2)
valid = torch.utils.data.TensorDataset(x_cv, y_cv)

# Create Data Loaders
train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid, batch_size=batch_size, shuffle=False)

train_loss = []
valid_loss = []

for epoch in range(n_epochs):
    # Set model to train configuration
    model.train()
    avg_loss = 0.  
    for i, (x, y) in enumerate(train_loader):
        # Predict/Forward Pass
        y_pred = model(x)
        # Compute loss
        loss = loss_fn(y_pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        avg_loss += loss.item() / len(train_loader)
    
    # Set model to validation configuration -Doesn't get trained here
    model.eval()        

    val_preds = np.zeros((len(x_cv),10))
    #y_pred_2 =[]
    y_true =[]
    preds = []
    labels = []
    for i, (x, y) in enumerate(valid_loader):
      y_pred_2=torch.argmax(model(x), dim = 1).tolist()
      y_pred_2 = map(int, y_pred_2)
      preds.extend(list(y_pred_2))
      labels.extend(y.tolist())
    
    val_accuracy = (np.sum(np.array(preds) == np.array(labels)))/len(preds)
    f1_scorE_sklearn = f1_score(preds,labels,average="weighted")
    print("f1 score is: ")
    print(f1_scorE_sklearn)
    print('Epoch {}/{} \t val_acc={:.4f} \t f1_score={:.4f}'.format(
                epoch + 1, n_epochs, val_accuracy, f1_scorE_sklearn))


f1 score is: 
0.5798898361924167
Epoch 1/8 	 val_acc=0.4239 	 f1_score=0.5799
f1 score is: 
0.6320072819199404
Epoch 2/8 	 val_acc=0.5345 	 f1_score=0.6320
f1 score is: 
0.6982476398356667
Epoch 3/8 	 val_acc=0.6561 	 f1_score=0.6982
f1 score is: 
0.7402584661117024
Epoch 4/8 	 val_acc=0.7151 	 f1_score=0.7403
f1 score is: 
0.7637332075485664
Epoch 5/8 	 val_acc=0.7482 	 f1_score=0.7637
f1 score is: 
0.7875953450007533
Epoch 6/8 	 val_acc=0.7761 	 f1_score=0.7876
f1 score is: 
0.7958987888205687
Epoch 7/8 	 val_acc=0.7893 	 f1_score=0.7959
f1 score is: 
0.8032176691570134
Epoch 8/8 	 val_acc=0.7965 	 f1_score=0.8032
